In [ ]:
# The params below are needed if the researcher wants to run the notebook in isolation with papermill
# query_function = None
# dataset_path = None
# result_file = None

In [1]:
import dask
import dask.dataframe as dd
import dask.array as da
import itertools as it
import numpy as np
import pandas as pd
import pyarrow
import numba
import cython
from time import time
import os
import matplotlib.pyplot as plt
import gc
import numpy as np
import json
import resource
import numpy as np

indices = ['filename', 'application', 'io_zone', 'redundancy_type']
combinations = list(it.combinations(indices, r=2))[:5]

# Function to log results to a file
def log_results_to_file(result_file, time, memory):
    with open(result_file, 'a') as f:
        f.write(f"Times: {time}, Memory: {memory}\n")
    
    # Check if the file was created
    if os.path.exists(result_file):
        print(f"File {result_file} created/updated successfully!")
    else:
        print(f"Error: {result_file} was not created.")

def base_medium_pandas_queries(log_dir, result_file):
    print(f",base_medium_pandas_queries")
    df = pd.read_csv(log_dir)
    query_index = 0
    for ix, iy in combinations:
        for col in ['request_io_size_bytes', 'disk_time']:
            query_index += 1 
            t1 = time()
            res = df.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum()
            time_elapsed = time() - t1
            memory_usage = res.memory_usage(deep=True).sum()

            # Store the time and memory usage for graphing later
            log_results_to_file(result_file, time_elapsed, memory_usage)

            print(f"medium,Q{query_index},{time_elapsed},{memory_usage}")
            

def base_medium_dask_queries(log_dir, result_file):
    print(f",base_medium_dask_queries")
    ddf = dd.read_csv(log_dir)
    query_index = 0
    for ix, iy in combinations:
        for col in ['request_io_size_bytes', 'disk_time']:
            query_index += 1 
            t1 = time()
            res = ddf.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum().compute()
            time_elapsed = time() - t1
            memory_usage = res.memory_usage(deep=True).sum()

            # Store the time and memory usage for graphing later
            log_results_to_file(result_file, time_elapsed, memory_usage)

            print(f"medium,Q{query_index},{time_elapsed},{memory_usage}")

def iomax_medium_pandas_queries(log_dir, result_file):
    print(f",iomax_medium_pandas_queries")
    df = pd.read_csv(log_dir)
    query_index = 0
    for ix, iy in combinations:
        for col in ['request_io_size_bytes', 'disk_time']:
            query_index += 1 
            t1 = time()
            m = 0
            if query_index == 1:
                x = df.groupby(indices).agg({'request_io_size_bytes': 'sum', 'disk_time': 'sum'})
                m = x.memory_usage(deep=True).sum()
            x.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum()
            time_elapsed = time() - t1

            # Store the time and memory usage for graphing later
            log_results_to_file(result_file, time_elapsed, m)

            print(f"medium,Q{query_index},{time_elapsed},{m}")

def iomax_medium_dask_queries(log_dir, result_file):
    print(f",iomax_medium_dask_queries")
    ddf = dd.read_csv(log_dir)
    query_index = 0
    for ix, iy in combinations:
        for col in ['request_io_size_bytes', 'disk_time']:
            query_index += 1 
            t1 = time()
            m = 0
            if query_index == 1:
                x = ddf.groupby(indices).agg({'request_io_size_bytes': 'sum', 'disk_time': 'sum'}).compute()
                m = x.memory_usage(deep=True).sum()
            x.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum()
            time_elapsed = time() - t1

            # Store the time and memory usage for graphing later
            log_results_to_file(result_file, time_elapsed, m)

            print(f"medium,Q{query_index},{time_elapsed},{m}")

iomax_medium_pandas_queries("datasets_thesios_io_traces/dataset-25m.csv", "results_medium_queries_csv/results_pandas_iomax_medium_25m.txt")

,iomax_medium_pandas_queries
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q1,8.7985360622406,131945601
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q2,1.7559683322906494,0
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q3,4.037119626998901,0
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q4,1.6657686233520508,0
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q5,1.7075364589691162,0
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q6,1.6552934646606445,0
File results_medium_queries_csv/results_pandas_iomax_medium_25m.txt created/updated successfully!
medium,Q7,0.07173347473144531,0
File results_medium_queries_csv/results_pandas_iomax_medium_25m